##**T5 Model**

Text-to-Text Transfer Transformer (T5) is a Transformer-based model using an encoder-decoder architecture. It is pretrained on a mix of unsupervised and supervised tasks, all converted into a text-to-text format. T5 excels in various sequence-to-sequence tasks like summarization and translation.

In this project, we will fine-tune the pretrained T5 model for the Abstractive Summarization task using Hugging Face Transformers and the XSum dataset from Hugging Face Datasets. We will use the T5 Base variant for its balanced performance and computational efficiency.

The T5 architecture consists of transformer encoder-decoder layers that process input text iteratively, capturing contextual information and providing meaningful representations. This structure allows for efficient information flow and hierarchical learning, enabling T5 to achieve state-of-the-art results across multiple NLP benchmarks while maintaining a simple and scalable design.

## T5 base Architecture


<center>
    <img src = "https://cdn.analyticsvidhya.com/wp-content/uploads/2024/05/Screenshot-318.png" width = 300, height = 450>
<p style = "font-size: 14px;
            font-family: 'Georgia', serif;
            text-align: center;
            margin-top: 10px;">T5-Base architecture. <br>Source: <a href = "https://www.analyticsvidhya.com/blog/2024/05/text-summarization-using-googles-t5-base/">Analytics Vidhya</a></p>
</center>

## Dependencies


Installations and Setup

First, install the necessary libraries and import the required modules:

In [ ]:
!pip install transformers                                     # Installing the transformers library (https://huggingface.co/docs/transformers/index)

In [13]:
from transformers import T5ForConditionalGeneration, T5Tokenizer         # T5 Tokenizer and architecture
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer        # These will help us to fine-tune our model
from transformers import DataCollatorForSeq2Seq                          # DataCollator to batch the data

In [2]:
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

Install the datasets library from Hugging Face in the enviornment.

In [3]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 2

In [4]:
# !pip install --upgrade datasets

In [5]:
# !pip install --upgrade pyarrow

In [6]:
# !pip install --upgrade pyarrow datasets

## Load dataset

Use the Hugging Face Datasets library to download the data we need to use for training, evaluation and testing. This can be easily done with the load_dataset function.

In [1]:
from datasets import load_dataset

In [2]:
# Load a subset of XSum dataset
# Load a small portion of the training and validation datasets
train_dataset = load_dataset("xsum", split="train[:1%]")
valid_dataset = load_dataset("xsum", split="validation[:1%]")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for xsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/xsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11332 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11334 [00:00<?, ? examples/s]

In [3]:
test_dataset = load_dataset("xsum", split="test[:1%]")

In [4]:
test_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 113
})

## Extreme Summarization (XSum) dataset
This dataset contains BBC articles along with single-sentence summaries. Each article includes an introductory sentence, professionally written by the article's author. The dataset comprises 226,711 articles, split into training (90%, 204,045), validation (5%, 11,332), and test (5%, 11,334) sets.

In case, we have sufficient compuational facility, we can load whole XSUM dataset using the following code and train our model on the same.

In [5]:
# dataset = load_dataset("EdinburghNLP/xsum")

# # Split the dataset into train, validation, and test sets
# train_dataset = dataset['train']
# valid_dataset = dataset['validation']
# test_dataset = dataset['test']

In [6]:
train_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 2040
})

In [7]:
# Print the first 5 examples in the training dataset
for i in range(5):
    print(train_dataset[i])

{'document': 'The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally apprecia

We can observe that the whole data is not properly readable. For better readability, let us convert this to DataFrme and see the data properly.

# Converting dataset to a Pandas DataFrame
We can convert the dataset using Pandas into DataFrame so that we can see the first filw enties of dataset in better way.

In [8]:
import pandas as pd

# Set Pandas options to display the full text in each cell
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)

# Convert to a Pandas DataFrame
df = pd.DataFrame(train_dataset)
df.head(5)

document  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  The full cost of damage in Newton Stewart, one of the areas worst affected, is still being assessed.\nRepair work is ongoing in Hawick and many roads in Peeblesshire remain badly affected by standing water.\nTrains on the west coast mainline face disruption due to damage at the Lamington Viaduct.\nMany businesses and householders were affected by flooding in Newton Stewart after the River Cree overflowed into the town.\nFirst Minister Nicola Sturgeon visited the area to inspect the damage.\nThe waters breached a retaining wall, flooding many commercial properties on Victoria Street - the main shopping thoroughfare.\nJeanette Tate, who owns the Cinnamon Cafe which was badly affected, said she could not fault the multi-agency response once the flood hit.\nHowever, she said more preventative work could have been carried out to ensure the retaining wall did not fail.\n"It is difficult but I do think there is so much publicity for Dumfries and the Nith - and I totally appreciate that - but it is almost like we're neglected or forgotten," she said.\n"That may not be true but it is perhaps my perspective over the last few days.\n"Why were you not ready to help us a bit more when the warning and the alarm alerts had gone out?"\nMeanwhile, a flood alert remains in place across the Borders because of the constant rain.\nPeebles was b

The dataset has the following fields:

document: the original BBC article to be summarized

summary: the single sentence summary of the BBC article

id: ID of the document-summary pair

In [9]:
df.describe()

,document,summary,id
count,2040,2040,2040
unique,2039,2038,2040
top,"This breaking news story is being updated and more details will be published shortly. Please refresh the page for the fullest version.\nIf you want to receive Breaking News alerts via email, or on a smartphone or tablet via the BBC News App then details on how to do so are available on this help page. You can also follow @BBCBreaking on Twitter to get the latest alerts.",All pictures are copyrighted.,39847414
freq,2,2,1


In [10]:
valid_dataset

Dataset({
    features: ['document', 'summary', 'id'],
    num_rows: 113
})

In [11]:
# Convert to a Pandas DataFrame
valid_df = pd.DataFrame(valid_dataset)
valid_df.head(5)

,document,summary,id
0,"The ex-Reading defender denied fraudulent trading charges relating to the Sodje Sports Foundation - a charity to raise money for Nigerian sport.\nMr Sodje, 37, is jointly charged with elder brothers Efe, 44, Bright, 50 and Stephen, 42.\nAppearing at the Old Bailey earlier, all four denied the offence.\nThe charge relates to offences which allegedly took place between 2008 and 2014.\nSam, from Kent, Efe and Bright, of Greater Manchester, and Stephen, from Bexley, are due to stand trial in July.\nThey were all released on bail.",Former Premier League footballer Sam Sodje has appeared in court alongside three brothers accused of charity fraud.,38295789
1,"Voges was forced to retire hurt on 86 after suffering the injury while batting during the County Championship draw with Somerset on 4 June.\nMiddlesex hope to have the Australian back for their T20 Blast game against Hampshire at Lord's on 3 August.\nThe 37-year-old has scored 230 runs in four first-class games this season at an average of 57.50.\n""Losing Adam is naturally a blow as he contributes significantly to everything we do,"" director of cricket Angus Fraser said.\n""His absence, however, does give opportunities to other players who are desperate to play in the first XI.\n""In the past we have coped well without an overseas player and I expect us to do so now.""\nDefending county champions Middlesex are sixth in the Division One table, having drawn all four of their matches this season.\nVoges retired from international cricket in February with a Test batting average of 61.87 from 31 innings, second only to Australian great Sir Donald Bradman's career average of 99.94 from 52 Tests.",Middlesex batsman Adam Voges will be out until August after suffering a torn calf muscle in his right leg.,40202028
2,"Seven photographs taken in the Norfolk countryside by photographer Josh Olins will appear in the June edition.\nIn her first sitting for a magazine, the duchess is seen looking relaxed and wearing casual clothes.\nThe shoot was in collaboration with the National Portrait Gallery, where two images are being displayed in the Vogue 100: A Century of Style exhibition.\nThe duchess, who has a keen interest in photography, has been patron of the National Portrait Gallery since 2012.\nNicholas Cullinan, director of the National Portrait Gallery, said: ""Josh has captured the duchess exactly as she is - full of life, with a great sense of humour, thoughtful and intelligent, and in fact, very beautiful.""\nHe said the images also encapsulated what Vogue had done over the past 100 years - ""to pair the best photographers with the great personalities of the day, in order to reflect broader shifts in culture and society"".\nAlexandra Shulman, editor-in-chief of British Vogue, said: ""To be able to publish a photographic shoot with the Duchess of Cambridge has been one of my greatest ambitions for the magazine.""\nThe collaboration for the June edition had resulted in ""a true celebration of our centenary as well as a fitting tribute to a young woman whose interest in both photography and the countryside is well known"", she said.\nOther royal portraits to have featured in the fashion magazine include Diana, Princess of Wales - who graced the cover four times - and Princess Anne.\nThe duchess is to visit the exhibition at the National Portrait Gallery on Wednesday, Kensington Palace said.",The Duchess of Cambridge will feature on the cover of British Vogue to mark the magazine's centenary.,36177725
3,"Chris Poole - known as ""moot"" online - created the site in 2003.\nIt has gone on to be closely associated with offensive and often illegal activity, including instances where the images of child abuse were shared.\nIt was widely credited as being the first place where leaked images of nude celebrities were posted following 2014's well-publicised security breach affecting Apple's iCloud service. That incident prompted a policy change on the site.\nHowever, 4chan ha

## Defining the model and tokenizer
Now, we download the pretrained model and fine-tune it.

In [14]:
# Initialize the model and tokenizer
model_name = "t5-base"  # You can use t5-small, t5-base, t5-large, etc.
model = T5ForConditionalGeneration.from_pretrained(model_name)
tokenizer = T5Tokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [15]:
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.9 MB/s eta 0:00:00


In [16]:
import evaluate                                            # Hugging Face's library for model evaluation

## Preprocessing

Before we can feed those texts to our model, we need to pre-process them and get them ready for the task. This is done by a Hugging Face Transformers Tokenizer which will tokenize the inputs (including converting the tokens to their corresponding IDs in the pretrained vocabulary) and put it in a format the model expects, as well as generate the other inputs that model requires.

In [17]:
# Preprocess function
def preprocess_function(examples):
    inputs = examples["document"]
    targets = examples["summary"]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [18]:
# Apply preprocessing to all splits
train_dataset = train_dataset.map(preprocess_function, batched=True)
valid_dataset = valid_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/2040 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/113 [00:00<?, ? examples/s]

In [19]:
test_dataset = test_dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/113 [00:00<?, ? examples/s]

For training Sequence to Sequence models, we need a special kind of data collator, which will not only pad the inputs to the maximum length in the batch, but also the labels. Thus, we use the DataCollatorForSeq2Seq provided by the Hugging Face Transformers library on our dataset.

DataCollatorForSeq2Seq is used to batch the data and these data collators may also automatically apply some processing techniques, such as padding.

In [21]:
# Instantiating Data Collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [22]:
# # Verify installation
# import transformers
# import accelerate

# print(f"transformers version: {transformers.__version__}")
# print(f"accelerate version: {accelerate.__version__}")

In [23]:
# !pip install transformers==4.42.3
# !pip install accelerate==0.21.0

## Defining parameters for training

We now use the Seq2SeqTrainingArguments class to set some relevant settings for fine-tuning. I will first define a directory to serve as output, and then define the evaluation strategy, learning rate, etc.

In [32]:
# Define the training arguments
seed = 42

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    seed = seed,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=4,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,                      # Enable mixed precision training
)

In [33]:
print(training_args)

Seq2SeqTrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
batch_eval_metrics=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=True,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
dispatch_batches=None,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_steps=None,
eval_strategy=epoch,
evaluation_strategy=None,
fp16=True,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[]

## Creating the Trainer Instance
Now, create a Seq2SeqTrainer instance to manage the training and evaluation process for our T5 model. The Seq2SeqTrainer class is specifically designed for sequence-to-sequence tasks, like text summarization. This instance is initialized with the model, training arguments, training and validation datasets, tokenizer, and data collator, which together facilitate efficient model training and evaluation.

In [34]:
# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

## Starting the Training Process
Initiate the training process using the train method of the Seq2SeqTrainer instance. This method will train the T5 base model on the provided training and validation datasets, applying the specified training arguments and configurations.

In [35]:
# Start training
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.915200,0.476516
2,0.504400,0.469751
3,0.468200,0.470956
4,0.445800,0.472782


TrainOutput(global_step=2040, training_loss=0.5809187861049876, metrics={'train_runtime': 1030.684, 'train_samples_per_second': 7.917, 'train_steps_per_second': 1.979, 'total_flos': 4969096386969600.0, 'train_loss': 0.5809187861049876, 'epoch': 4.0})


##Evaluating and Saving Model

After training and validating the model, we now evaluate its performance on the test dataset. We will use the evaluate method for this purpose.

In [36]:
# Evaluating model performance on the test dataset
test = trainer.evaluate(eval_dataset = test_dataset)
print(test) # Printing results

{'eval_loss': 0.49926337599754333, 'eval_runtime': 3.6732, 'eval_samples_per_second': 30.763, 'eval_steps_per_second': 7.895, 'epoch': 4.0}


In [37]:
# Saving model to a custom directory
directory = "T5_finetuned_xsum_subset"
trainer.save_model(directory)

# Saving model tokenizer
tokenizer.save_pretrained(directory)

('T5_finetuned_xsum_subset/tokenizer_config.json',
 'T5_finetuned_xsum_subset/special_tokens_map.json',
 'T5_finetuned_xsum_subset/spiece.model',
 'T5_finetuned_xsum_subset/added_tokens.json')

##Inference

We will now perform inference using the model we trained on an arbitrary article. For this, we'll utilize the pipeline method from Hugging Face Transformers, which offers various pipelines for different tasks. For our purpose, we will use the summarization pipeline.

The pipeline method requires the trained model and tokenizer as inputs.

In [39]:
from transformers import pipeline

summarizer = pipeline("summarization", model=model, tokenizer=tokenizer)

summarizer(
    test_dataset[0]["document"],
    min_length=64,
    max_length=128,
)

[{'summary_text': 'Prison leavers in Wales are living rough for up to a year before finding suitable accommodation, a charity says . "there\'s a desperate need for it," says a worker who has been jailed for 20 years for burglary offences . 20,000 new affordable homes will be built in the next five years, the government says.'}]

In [41]:
examples = [
    test_dataset[i]["document"]
    for i in range(5)
]

In [75]:
import pickle

with open('Summarization_examples.pkl', 'wb') as f: pickle.dump(examples, f)
with open('Summarization_examples.pkl', 'rb') as f: loaded_examples = pickle.load(f)

In [76]:
from google.colab import files
files.download("Summarization_examples.pkl")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [44]:
loaded_examples[0]

'Prison Link Cymru had 1,099 referrals in 2015-16 and said some ex-offenders were living rough for up to a year before finding suitable accommodation.\nWorkers at the charity claim investment in housing would be cheaper than jailing homeless repeat offenders.\nThe Welsh Government said more people than ever were getting help to address housing problems.\nChanges to the Housing Act in Wales, introduced in 2015, removed the right for prison leavers to be given priority for accommodation.\nPrison Link Cymru, which helps people find accommodation after their release, said things were generally good for women because issues such as children or domestic violence were now considered.\nHowever, the same could not be said for men, the charity said, because issues which often affect them, such as post traumatic stress disorder or drug dependency, were often viewed as less of a priority.\nAndrew Stevens, who works in Welsh prisons trying to secure housing for prison leavers, said the need for acc

##Saving the model

In [63]:
best_model_save_path = f"Fine_tuned_T5_XSum"
model.save_pretrained(best_model_save_path)

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'min_length': 30, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3}


In [64]:
tokenizer_save_path = "Tokenizer"
tokenizer.save_pretrained(tokenizer_save_path)

('Tokenizer/tokenizer_config.json',
 'Tokenizer/special_tokens_map.json',
 'Tokenizer/spiece.model',
 'Tokenizer/added_tokens.json')

## Compressing the model

In [65]:
zip_file_path, source_path = f"{best_model_save_path}.zip", best_model_save_path

In [66]:
import os
# Zipping the model to download
os.system(f"zip -r {zip_file_path} {source_path}")

0

In [80]:
os.system(f"zip -r Tokenizer.zip tokenizer")

0

## Download the model

In [77]:
# Downloading the zipped model into the local
from google.colab import files

In [78]:
files.download(zip_file_path)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [69]:
from google.colab import drive
drive.mount("drive")

Drive already mounted at drive; to attempt to forcibly remount, call drive.mount("drive", force_remount=True).


In [70]:
print(os.getcwd())

/content


In [72]:
!cp -r /content/Fine_tuned_T5_XSum /content/drive/MyDrive/DS_Projects/Text_Summarization_using_T5_XSUM

In [81]:
files.download("Tokenizer.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [82]:
!cp -r /content/Tokenizer /content/drive/MyDrive/DS_Projects/Text_Summarization_using_T5_XSUM